Modern Deep Convolutional Neural Networks with Pytorch

Multi-Layer Perceptron Classifier

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 
import torch.backends.cudnn as cudnn 

import torchvision 
import torchvision.transforms as transforms 

import matplotlib.pyplot as plt 
import numpy as np 

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor();
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=True, num_workers=2)




classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
def imshow(img):
    plt.figure(figsize=(10, 10))
    img = img / 2 + 0.5 #unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()



# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

#show images
imshow(torchvision.utils.make_grid(images))

#print labels 
print(''.join(%5s % classes[labels[j]] for j in range(4)))

In [ ]:
class MLP(nn.Module):
    def __init__(self, n_input_neurons, n_output_neurons):
        super().__init__()

        self.main = nn.Sequential( # sequential allows us to write several layers as a sequence
            nn.Linear(n_input_neurons, 256),
            nn.LeakyReLU(negative_slope=0.2, inplace=True) #non-linearity
            nn.Linear(256, 128),
            nn.LeakyReLU(negative_slope=0.2, inplace=True) #non-linearity
            nn.Linear(128, n_output_neurons)

        )
         


    def forward(self, x): #feed forward network
        # x has shappe [batch_size, 3, height, width]
        b_size = x.shape[0]
        x = x.view(b_size, -1)

        return self.main(x)

In [ ]:
height, width = images.shape0[2: ]

In [ ]:
# Neural Network
net = MLP(n_input_neurons = 3*height*width,  n_output_neurons = 10).cuda()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
# Training loop
for epoch in range(5): # loop over the dataset mulitple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):

        # get the inputs
        images, labels = data
        images, labels = images.cuda(), labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward pass + backward propagation + optimize
        output = net(images)
        loss = criterion(output, labels)

        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.items()
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print(' [%d, %5d] loss: %.3f' %
             (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
def get_quality(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader: 
            images, labels = data 
            images, labels = images.cuda(), labels.cuda()
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        print('Accuracy of the network on test images: %d %%' % (100 * correct / total))


In [ ]:
get_quality(net)